- https://changkun.de/modern-cpp

- `nullptr`, 将`NULL`定义为0会导致重载混乱

```cpp
void foo(char*);
void foo(int);

foo(NULL);
```

- `constexpr`, C14开始, `constexpr`函数可以在内部使用局部变量, 循环, 分支等简单语句??

```cpp
constexpr int fibonacci(const int n) 
{
    return n == 1 || n == 2 ? 1 : fibonacci(n-1) + fibonacci(n-2);
}
```

- C17后可以在`if`和`switch`语句中声明临时变量(C# `using`??)

- C17结构化绑定

```cpp
std::tuple<int, double, std::string> f() 
{
    return std::make_tuple(1, 2.3, "456");
}

int main() 
{
    auto [x, y, z] = f();
    std::cout << x << ", " << y << ", " << z << std::endl;
    return 0;
}
```

- trailing return type

```cpp
template<typename T, typename U>
auto add2(T x, U y) -> decltype(x+y)
{
    return x + y;
}

template<typename T, typename U>
auto add3(T x, U y)
{
    return x + y;
}
```

- `if constexpr`

```cpp
template<typename T>
auto print_type_info(const T& t) 
{
    if constexpr (std::is_integral<T>::value) 
    {
        return t + 1;
    } 
    else 
    {
        return t + 0.001;
    }
}
int main() 
{
    std::cout << print_type_info(5) << std::endl;
    std::cout << print_type_info(3.14) << std::endl;
}
```

- `using`

```cpp
typedef int (*process)(void *);
using NewProcess = int(*)(void *);
```

- 变长参数模板

```cpp
template<typename... Ts>
void magic(Ts... args) 
{
    std::cout << sizeof...(args) << std::endl;
}

template<typename T0>
void printf1(T0 value) 
{
    std::cout << value << std::endl;
}
template<typename T, typename... Ts>
void printf1(T value, Ts... args) 
{
    std::cout << value << std::endl;
    printf1(args...);
}
int main() 
{
    printf1(1, 2, "123", 1.1);
    return 0;
}

template<typename T0, typename... T>
void printf2(T0 t0, T... t) 
{
    std::cout << t0 << std::endl;
    if constexpr (sizeof...(t) > 0) printf2(t...);
}

template<typename T, typename... Ts>
auto printf3(T value, Ts... args) 
{
    std::cout << value << std::endl;
    (void) std::initializer_list<T>
    	{([&args] 
    		{
        		std::cout << args << std::endl;
    		}(), value)...
    	};
}

#include <iostream>
template<typename ... T>
auto sum(T ... t) 
{
    return (t + ...);
}
int main() 
{
    std::cout << sum(1, 2, 3, 4, 5, 6, 7, 8, 9, 10) << std::endl;
}
```

- 继承构造

```cpp
class Base 
{
public:
    int value1;
    int value2;
    Base() 
    {
        value1 = 1;
    }
    Base(int value) : Base() 
    { // 委托 Base() 构造函数
        value2 = value;
    }
};
class Subclass : public Base 
{
public:
    using Base::Base; // 继承构造
};
int main() 
{
    Subclass s(3);
    std::cout << s.value1 << std::endl;
    std::cout << s.value2 << std::endl;
}
```

- Argument Passing and Naming Conventions
	- https://docs.microsoft.com/en-us/cpp/cpp/argument-passing-and-naming-conventions?view=msvc-160
	- Not all conventions are available on all supported platforms, and some conventions use platform-specific implementations. In most cases, keywords or compiler switches that specify an unsupported convention on a particular platform are ignored, and the platform default convention is used.
	- On x86 plaftorms, all arguments are widened to 32 bits when they are passed. Return values are also widened to 32 bits and returned in the EAX register, except for 8-byte structures, which are returned in the EDX:EAX register pair. Larger structures are returned in the EAX register as pointers to hidden return structures. Parameters are pushed onto the stack from right to left. Structures that are not PODs will not be returned in registers.
	- Keyword | Stack cleanup | Parameter passing
	  --|--|--
	  __cdecl | Caller | Pushes parameters on the stack, in reverse order (right to left)
	  _clrcall | n/a | Load parameters onto CLR expression stack in order (left to right)
	  _stdcall | Callee | Pushes parameters on the stack, in reverse order (right to left)
	  _fastcall | Callee | Stored in registers, then pushed on stack
	  _thiscall | Callee | Pushed on stack; this pointer stored in ECX
	  _vectorcall | Callee | Stored in registers, then pushed on stack in reverse order (right to left)
	- __cdecl is the default calling convention for C and C++ programs. Because the stack is cleaned up by the caller, it can do vararg functions. The __cdecl calling convention creates larger executables than __stdcall, because it requires each function call to include stack cleanup code. 
	- __clrcall specifies that a function can only be called from managed code. Use __clrcall for all virtual functions that will only be called from managed code. However this calling convention cannot be used for functions that will be called from native code. The __clrcall modifier is Microsoft-specific.
	- The __stdcall calling convention is used to call Win32 API functions. The callee cleans the stack, so the compiler makes vararg functions __cdecl. Functions that use this calling convention require a function prototype. The __stdcall modifier is Microsoft-specific.
	- The __fastcall calling convention specifies that arguments to functions are to be passed in registers, when possible. This calling convention only applies to the x86 architecture.
	- The Microsoft-specific __thiscall calling convention is used on C++ class member functions on the x86 architecture. It's the default calling convention used by member functions that don't use variable arguments (vararg functions).
	- The __vectorcall calling convention specifies that arguments to functions are to be passed in registers, when possible. __vectorcall uses more registers for arguments than __fastcall or the default x64 calling convention use. The __vectorcall calling convention is only supported in native code on x86 and x64 processors that include Streaming SIMD Extensions 2 (SSE2) and above. Use __vectorcall to speed functions that pass several floating-point or SIMD vector arguments and perform operations that take advantage of the arguments loaded in registers. 